# Agentes en Langchain

## Prompt

In [8]:
# from langchain import hub

# Get the prompt to use - you can modify this!
# prompt = hub.pull("dialogue_agent")
# prompt.messages

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Respond only in Spanish."),
        MessagesPlaceholder(variable_name='chat_history', optional=True),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name='agent_scratchpad')
    ]
)

## Model

In [4]:
%cd ..

c:\Users\Chris\Desktop\updt\Programacion\ia projects\MultiAgent-SecondBrain-AI


C:\Users\Chris\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
from src.utils.open_router import ChatOpenRouter

model_name = 'openchat/openchat-7b:free'

model = ChatOpenRouter(model_name)

## Tool

In [15]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Retorna el tamañño del string 'word'"""
    return len(word)

tools = []

## Agent

In [16]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(model, tools, prompt)

## AgentExecutor

In [17]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)

In [32]:
agent_executor.invoke({"input": "Hola!"})

{'input': 'Hola!', 'output': 'Hola! ¿En qué puedo ayudarte hoy?'}

In [33]:
agent_executor.invoke({"input": "Quiero que me cuentes un chiste"})

{'input': 'Quiero que me cuentes un chiste',
 'output': '¿Sabes por qué los días son tan largos? Es porque el sol se levanta temprano. ¡Jajaja!'}

### Añadiendo memoria

In [34]:
agent_executor.invoke({"input": "Hola, mi nombre es Christian!", "chat_history": []})

{'input': 'Hola, mi nombre es Christian!',
 'chat_history': [],
 'output': 'Hola Christian, un placer conocerte.'}

In [35]:
from langchain_core.messages import AIMessage, HumanMessage

agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="Quiero que recuerdes mi nombre: Christian!"),
            AIMessage(content="Hola Christian! En que puedo ayudarte?"),
        ],
        "input": "Cual es mi nombre?",
    }
)

{'chat_history': [HumanMessage(content='Quiero que recuerdes mi nombre: Christian!'),
  AIMessage(content='Hola Christian! En que puedo ayudarte?')],
 'input': 'Cual es mi nombre?',
 'output': 'Tu nombre es Christian. ¿En qué puedo ayudarte?'}

### RunnableWithMessageHistory

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [18]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

In [19]:
agent_with_chat_history.invoke(
    {"input": "Hola! soy Christian"},
    config={"configurable": {"session_id": "1"}},
)

{'input': 'Hola! soy Christian',
 'chat_history': [HumanMessage(content='Hola! soy Christian'),
  AIMessage(content='Hola Christian! ¿En qué puedo ayudarte hoy?'),
  HumanMessage(content='Cual es mi nombre?'),
  AIMessage(content='Tu nombre es Christian.')],
 'output': 'Hola Christian! ¿En qué puedo ayudarte hoy?'}

In [20]:
agent_with_chat_history.invoke(
    {"input": "Cual es mi nombre?"},
    config={"configurable": {"session_id": "1"}},
)

{'input': 'Cual es mi nombre?',
 'chat_history': [HumanMessage(content='Hola! soy Christian'),
  AIMessage(content='Hola Christian! ¿En qué puedo ayudarte hoy?'),
  HumanMessage(content='Cual es mi nombre?'),
  AIMessage(content='Tu nombre es Christian.'),
  HumanMessage(content='Hola! soy Christian'),
  AIMessage(content='Hola Christian! ¿En qué puedo ayudarte hoy?')],
 'output': 'Tu nombre es Christian.'}

### Memoria para varios usuarios

In [47]:
from langchain_core.runnables import ConfigurableFieldSpec
from langchain_community.chat_message_histories import SQLChatMessageHistory



def get_session_history(user_id: str, conversation_id: str):
    return SQLChatMessageHistory(f"{user_id}--{conversation_id}", "sqlite:///memory.db")

runnable = prompt | model

with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id",
            annotation=str,
            name="Conversation ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
)

In [48]:
with_message_history.invoke(
    {"input": "Hola, mi nombre es Christian!"},
    config={"configurable": {"user_id": "1", "conversation_id": "1"}},
)

AIMessage(content='Hola Christian, un gusto hablar contigo. ¿En qué puedo ayudarte hoy?', response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 128, 'total_tokens': 153}, 'model_name': 'openchat/openchat-7b:free', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-22d702f3-46c6-4e81-aef6-6ce3a7829baa-0', usage_metadata={'input_tokens': 128, 'output_tokens': 25, 'total_tokens': 153})

In [49]:
with_message_history.invoke(
    {"input": "Cual es mi nombre?"},
    config={"configurable": {"user_id": "1", "conversation_id": "1"}},
)

AIMessage(content='Tu nombre es Christian, como mencionaste al inicio. ¿En qué puedo ayudarte hoy?', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 174, 'total_tokens': 200}, 'model_name': 'openchat/openchat-7b:free', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-44cc893a-fe92-4fb2-a6d5-cc1e8c587168-0', usage_metadata={'input_tokens': 174, 'output_tokens': 26, 'total_tokens': 200})

In [50]:
with_message_history.invoke(
    {"input": "Cual es mi nombre?"},
    config={"configurable": {"user_id": "1", "conversation_id": "2"}},
)

AIMessage(content='Lo siento, aún no tengo información sobre su nombre. Si me proporciona más detalles, estaré encantado de ayudarle en lo que pueda.', response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 101, 'total_tokens': 145}, 'model_name': 'openchat/openchat-7b:free', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-506db4c1-d826-4caf-afe8-fb8cc729bca4-0', usage_metadata={'input_tokens': 101, 'output_tokens': 44, 'total_tokens': 145})

## LangGraph

In [52]:
%pip install -U langgraph

   ---------------------------------------- 0.0/102.5 kB ? eta -:--:--
   ----------- ---------------------------- 30.7/102.5 kB 1.4 MB/s eta 0:00:01
   ----------- ---------------------------- 30.7/102.5 kB 1.4 MB/s eta 0:00:01
   -------------------------------------- 102.5/102.5 kB 738.0 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [53]:
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import create_react_agent

system_message = "You are a helpful assistant. Respond only in Spanish."

agent = create_react_agent(model, tools, messages_modifier=system_message)

query = "Hola, mi nombre es Christian!"

messages = agent.invoke({"messages": [("user", query)]})

C:\Users\Chris\AppData\Local\Temp\ipykernel_4876\1525963175.py:6: LangGraphDeprecationWarning: Parameter 'messages_modifier' in function 'create_react_agent' is deprecated as of version 0.1.9 and will be removed in version 0.2.0. Use 'state_modifier' parameter instead.
  agent = create_react_agent(model, tools, messages_modifier=system_message)


In [54]:
messages

{'messages': [HumanMessage(content='Hola, mi nombre es Christian!', id='45efab64-bcde-4521-84eb-2851e02a3813'),
  AIMessage(content='Hola Christian, un gusto hablar contigo. ¿En qué puedo ayudarte hoy?', response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 43, 'total_tokens': 68}, 'model_name': 'openchat/openchat-7b:free', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6973bb6b-da9e-4803-9ab5-9cf471c75271-0', usage_metadata={'input_tokens': 43, 'output_tokens': 25, 'total_tokens': 68})]}

### Añadiendo Memoria

In [55]:
from langchain_core.messages import SystemMessage
from langgraph.checkpoint import MemorySaver

system_message = SystemMessage(content=system_message)

memory = MemorySaver()

agent = create_react_agent(model, tools, messages_modifier=system_message, checkpointer=memory)

config = {"configurable": {"thread_id": "test-thread"}}

C:\Users\Chris\AppData\Local\Temp\ipykernel_4876\2446920177.py:8: LangGraphDeprecationWarning: Parameter 'messages_modifier' in function 'create_react_agent' is deprecated as of version 0.1.9 and will be removed in version 0.2.0. Use 'state_modifier' parameter instead.
  agent = create_react_agent(model, tools, messages_modifier=system_message, checkpointer=memory)


In [56]:
print(
    agent.invoke(
        {
            "messages": [
                ("user", "Hola, mi nombre es Christian! Me contarías un chiste?")
            ]
        },
        config,
    )["messages"][-1].content
)
print("---")
print(
    agent.invoke({"messages": [("user", "Recuerdas mi nombre?")]}, config)["messages"][
        -1
    ].content
)
print("---")
print(
    agent.invoke({"messages": [("user", "Me repites la primera pregunta que te hice?")]}, config)[
        "messages"
    ][-1].content
)

Hola Christian! Aquí tienes un chiste en español:

¿Por qué no habla el zorro?
Porque le cae la cola al abrir la boca.

¡Espero que te haya hecho reír!
---
Sí, recuerdo que tu nombre es Christian. ¿En qué puedo ayudarte hoy?
---
Claro, aquí está la primera pregunta que hiciste:

¿Podrías contarme un chiste?

¡En serio! ¿En qué puedo ayudarte hoy? ¿Tienes alguna pregunta adicional o necesitas asistencia con algo en especial?
